<a href="https://colab.research.google.com/github/jkfip/Car_Recognition/blob/master/Car_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CAR RECOGNITION**

In [25]:
#upload car data from google drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
from google.colab import files
uploaded = files.upload()

Saving 00128.jpg to 00128 (1).jpg


In [27]:
#Unzip file from zip file
from zipfile import ZipFile
file_name="/content/drive/My Drive/car_data.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('finish')

finish


In [28]:
#import liberaries
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator,load_img
import pandas as pd
from keras.applications.vgg19 import VGG19
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np
import cv2
from PIL import Image
import numpy as np
from skimage import transform



In [29]:
#resize all image 
IMAGE_SIZE=[224,224]


In [30]:
#Import the Inception V3 liberary 

inception=InceptionV3(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)

In [31]:
#don't train existing weights
for layer in inception.layers:
  layer.trainable=False

In [32]:
#Flattening the layer
x=Flatten()(inception.output)


In [33]:
prediction=Dense(196,activation='softmax')(x)

**INCEPTION V3 MODEL**

In [34]:
#Create a model object
model1=Model(inputs=inception.input,outputs=prediction)

In [35]:
#view the structure of object of INCEPTION V3 MODEL
model1.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 111, 111, 32) 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 111, 111, 32) 0           batch_normalization_94[0][0]     
_______________________________________________________________________________________

In [36]:
#Complie the INCEPTION V3 MODEL
model1.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [37]:
#Use the Image Data Generator to import the images from dataset
train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1/.255)

In [38]:
#Reading all the  train data from flow_from_directory
training_set=train_datagen.flow_from_directory('/content/car_data/train',
                                              target_size=(224,224),
                                             batch_size=32,
                                              class_mode='categorical')

Found 8144 images belonging to 196 classes.


In [39]:
#Reading all the  train data from flow_from_directory
test_set=test_datagen.flow_from_directory('/content/car_data/test',
                                              target_size=(224,224),
                                             batch_size=32,
                                              class_mode='categorical')

Found 8041 images belonging to 196 classes.


In [40]:
#fit the INCEPTION V3 MODEL
model1_accuracy=model1.fit(training_set,
                      validation_data=test_set,
                      epochs=3,
                      steps_per_epoch=len(training_set),
                       validation_steps=len(test_set)
                       )

Epoch 1/3
255/255 [==============================] - 205s 804ms/step - loss: 29.7524 - accuracy: 0.0922 - val_loss: 4336.6548 - val_accuracy: 0.0062
Epoch 2/3
255/255 [==============================] - 200s 784ms/step - loss: 18.4964 - accuracy: 0.2921 - val_loss: 5439.4956 - val_accuracy: 0.0066
Epoch 3/3
255/255 [==============================] - 199s 781ms/step - loss: 15.4747 - accuracy: 0.3991 - val_loss: 7004.3682 - val_accuracy: 0.0061


In [41]:

from keras.preprocessing import image

img_width, img_height = 224, 224
img = image.load_img('/content/car_data/train/Acura Integra Type R 2001/00198.jpg', target_size = (img_width, img_height))
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)

predict1=model1.predict(img)

In [42]:
np.argmax(predict1)

159

**ALEXNET MODEL**

In [43]:

#Instantiate an empty model
model2 = Sequential()

# 1st Convolutional Layer
model2.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11), strides=(4,4), padding='valid'))
model2.add(Activation('relu'))
# Max Pooling
model2.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
model2.add(BatchNormalization())

# 2nd Convolutional Layer
model2.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model2.add(Activation('relu'))
# Max Pooling
model2.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
model2.add(BatchNormalization())

# 3rd Convolutional Layer
model2.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model2.add(Activation('relu'))
model2.add(BatchNormalization())

# 4th Convolutional Layer
model2.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model2.add(Activation('relu'))
model2.add(BatchNormalization())

# 5th Convolutional Layer
model2.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model2.add(Activation('relu'))
# Max Pooling
model2.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
model2.add(BatchNormalization())

# Passing it to a Fully Connected layer
model2.add(Flatten())
# 1st Fully Connected Layer
model2.add(Dense(4096, input_shape=(224*224*3,)))
model2.add(Activation('relu'))
# Add Dropout to prevent overfitting
model2.add(Dropout(0.4))
model2.add(BatchNormalization())

# 2nd Fully Connected Layer
model2.add(Dense(4096))
model2.add(Activation('relu'))
# Add Dropout
model2.add(Dropout(0.4))
model2.add(BatchNormalization())

# 3rd Fully Connected Layer
model2.add(Dense(1000))
model2.add(Activation('relu'))
# Add Dropout
model2.add(Dropout(0.4))
model2.add(BatchNormalization())

# Output Layer
model2.add(Dense(196))
model2.add(Activation('softmax'))

In [44]:
#view the structure of object of INCEPTION V3 MODEL
model2.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_188 (Conv2D)          (None, 54, 54, 96)        34944     
_________________________________________________________________
activation_188 (Activation)  (None, 54, 54, 96)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization_188 (Bat (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_189 (Conv2D)          (None, 17, 17, 256)       2973952   
_________________________________________________________________
activation_189 (Activation)  (None, 17, 17, 256)       0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 8, 8, 256)         0

In [45]:
#Complie the ALEXNET MODEL
model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [46]:
#fit the ALEXNET MODEL
model2_accuracy=model2.fit(training_set,
                      validation_data=test_set,
                      epochs=10,
                      steps_per_epoch=len(training_set),
                       validation_steps=len(test_set)
                       )

Epoch 1/10
255/255 [==============================] - 191s 748ms/step - loss: 5.9307 - accuracy: 0.0081 - val_loss: 18869.1191 - val_accuracy: 0.0052
Epoch 2/10
255/255 [==============================] - 190s 744ms/step - loss: 5.7044 - accuracy: 0.0102 - val_loss: 13521.5391 - val_accuracy: 0.0058
Epoch 3/10
255/255 [==============================] - 189s 743ms/step - loss: 5.5812 - accuracy: 0.0133 - val_loss: 8656.6436 - val_accuracy: 0.0111
Epoch 4/10
255/255 [==============================] - 189s 741ms/step - loss: 5.4595 - accuracy: 0.0157 - val_loss: 7399.0786 - val_accuracy: 0.0090
Epoch 5/10
255/255 [==============================] - 189s 741ms/step - loss: 5.3736 - accuracy: 0.0208 - val_loss: 12009.1494 - val_accuracy: 0.0057
Epoch 6/10
255/255 [==============================] - 189s 740ms/step - loss: 5.3009 - accuracy: 0.0216 - val_loss: 12266.7705 - val_accuracy: 0.0065
Epoch 7/10
255/255 [==============================] - 190s 743ms/step - loss: 5.2157 - accuracy: 0.023

In [47]:

from keras.preprocessing import image

img_width, img_height = 224, 224
img = image.load_img('/content/car_data/train/Acura Integra Type R 2001/00198.jpg', target_size = (img_width, img_height))
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)

predict2=model2.predict(img)

In [48]:
np.argmax(predict2)

122

**VGG16 MODEL**

In [49]:
#Import the VGG16 liberary 
vgg16=VGG16(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)

58892288/58889256 [==============================] - 0s 0us/step


In [50]:
#don't train existing weights in VGG16 MODEL
for layer in vgg16.layers:
  layer.trainable=False

In [51]:
#Flattening the layer
x=Flatten()(vgg16.output)


In [52]:
prediction=Dense(196,activation='softmax')(x)

In [53]:
#Create a model object
model3=Model(vgg16.input,prediction)

In [54]:
#view the structure of object of VGG16 MODEL
model3.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [55]:
#Compile the VGG16 MODEL
model3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [56]:
#fit the VGG16 MODEL
model3_accuracy=model3.fit(training_set,
                      validation_data=test_set,
                      epochs=10,
                      steps_per_epoch=len(training_set),
                       validation_steps=len(test_set)
                       )

Epoch 1/10
255/255 [==============================] - 207s 811ms/step - loss: 6.4951 - accuracy: 0.0821 - val_loss: 1052.3928 - val_accuracy: 0.0662
Epoch 2/10
255/255 [==============================] - 206s 806ms/step - loss: 3.0401 - accuracy: 0.4133 - val_loss: 1176.5314 - val_accuracy: 0.0781
Epoch 3/10
255/255 [==============================] - 210s 824ms/step - loss: 2.0165 - accuracy: 0.5820 - val_loss: 1149.0974 - val_accuracy: 0.1083
Epoch 4/10
255/255 [==============================] - 215s 842ms/step - loss: 1.4299 - accuracy: 0.6812 - val_loss: 1278.2780 - val_accuracy: 0.1021
Epoch 5/10
255/255 [==============================] - 218s 857ms/step - loss: 1.0681 - accuracy: 0.7554 - val_loss: 1227.1877 - val_accuracy: 0.1244
Epoch 6/10
255/255 [==============================] - 220s 863ms/step - loss: 0.8226 - accuracy: 0.8038 - val_loss: 1177.3085 - val_accuracy: 0.1485
Epoch 7/10
255/255 [==============================] - 217s 852ms/step - loss: 0.7263 - accuracy: 0.8271 - 

In [57]:

from keras.preprocessing import image

img_width, img_height = 224, 224
img = image.load_img('/content/car_data/train/Acura Integra Type R 2001/00198.jpg', target_size = (img_width, img_height))
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)

predict3=model3.predict(img)

In [58]:

from keras.preprocessing import image

img_width, img_height = 224, 224
img = image.load_img('/content/car_data/train/Acura Integra Type R 2001/00198.jpg', target_size = (img_width, img_height))
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)

predict3=model3.predict(img)

In [59]:
np.argmax(predict3)

171

**VGG19 MODEL**

In [60]:

vgg19=VGG19(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)

80142336/80134624 [==============================] - 0s 0us/step


In [61]:
#don't train existing weights in VGG19 MODEL
for layer in vgg19.layers:
  layer.trainable=False

In [62]:
#Flattening the layer
x=Flatten()(vgg19.output)


In [63]:
prediction=Dense(196,activation='softmax')(x)

In [64]:
#Create a model object 
model4=Model(vgg19.input,prediction)

In [65]:
#view the structure of object of VGG19 MODEL
model4.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [66]:
#compile the VGG19 MODEL
model4.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [67]:
#fit the VGG19 MODEL
model4_accuracy=model4.fit(training_set,
                      validation_data=test_set,
                      epochs=10,
                      steps_per_epoch=len(training_set),
                       validation_steps=len(test_set)
                       )

Epoch 1/10
255/255 [==============================] - 224s 878ms/step - loss: 7.0235 - accuracy: 0.0686 - val_loss: 990.3709 - val_accuracy: 0.0627
Epoch 2/10
255/255 [==============================] - 222s 869ms/step - loss: 3.7098 - accuracy: 0.3475 - val_loss: 1005.8893 - val_accuracy: 0.0985
Epoch 3/10
255/255 [==============================] - 222s 872ms/step - loss: 2.5055 - accuracy: 0.5169 - val_loss: 1109.4481 - val_accuracy: 0.1021
Epoch 4/10
255/255 [==============================] - 219s 860ms/step - loss: 1.8233 - accuracy: 0.6186 - val_loss: 1155.1672 - val_accuracy: 0.1112
Epoch 5/10
255/255 [==============================] - 224s 879ms/step - loss: 1.3348 - accuracy: 0.7047 - val_loss: 1256.3066 - val_accuracy: 0.1116
Epoch 6/10
255/255 [==============================] - 225s 881ms/step - loss: 1.1989 - accuracy: 0.7381 - val_loss: 1334.1744 - val_accuracy: 0.1118
Epoch 7/10
255/255 [==============================] - 220s 864ms/step - loss: 1.0538 - accuracy: 0.7630 - v

In [68]:

from keras.preprocessing import image

img_width, img_height = 224, 224
img = image.load_img('/content/car_data/train/Acura Integra Type R 2001/00198.jpg', target_size = (img_width, img_height))
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)

predict4=model4.predict(img)

In [69]:
np.argmax(predict4)

84